In [108]:
import pandas as pd
import numpy as np

In [109]:
def load_data(file_name):
    return pd.read_excel(file_name,engine="openpyxl",header = None)

In [110]:
def pre_cleaning(df):
    #df.columns = ["job_name","link","3","4","5","6"]
    #df = df.fillna(value="Private Advertiser").rename(columns = {"3":"Company"})
    #df['ad_type'] = df['link'].str.extract('\=(\w+)\#')
    df['company_name']=df[2]
    df[['posted_time','featured_at']]=df[4].str.split(',',expand=True)[[0,2]]
    df.loc[df['company_name'].isna(),'company_name']= df['featured_at']
    df['ad_type']= df[1].str.extract('\=(\w+)\#')
    return df

In [111]:
def listing_time_cleaning(df):
    df['listing_time_indays'] = df['posted_time'].str.extract('(\d+[a-z])')
    df['listing_time_indays'] = df['listing_time_indays'].str.extract('(\w+)d')
    df['listing_time_indays'].fillna('0', inplace = True)
    df['listing_time_indays'] #notice that if the ad_type is promoted, the listing time is recorded as "Featured", which means the listing times of those featured advs may be longer.
    return df

In [112]:
#useful function
def deduplication(x):
    mid = len(x)//2
    if x != 'unknown':
        return x[:mid+1]
    else:
        return 'unknown'
def smart_deduplication(x):
    trim = x.strip()
    index = (trim+trim).find(trim,1)
    if index != -1:
        return trim[:index]
    else:
        return 'unknown'

In [113]:
def location_cleaning(df):
    df[['location','area']] = df[3].str.split("area:",expand= True)
    df['location'] = df['location'].apply(lambda x:x[9:])
    df['location'] = df['location'].apply(smart_deduplication)
    df['area'].fillna('unknown', inplace = True)
    df['area'] = df['area'].apply(smart_deduplication)
    return df

In [114]:
def classification_cleaning(df):
    df[['subclassification','Classification']] = df[5].str.split('subClassification:',expand=True)
    df['subclassification'] = df['subclassification'].apply(lambda x:x[15:])
    df['Classification'].fillna('unknown', inplace = True)
    df['Classification'] = df['Classification'].apply(smart_deduplication)
    df[['mainClassification','7','subclassification']] = df['subclassification'].str.split('&',expand=True)
    df['subclassification'].fillna('unknown', inplace = True)
    return df

In [115]:
def random_salary(df,salary_floor,salary_ceiling):    
    df['salary']=df[5]
    df[df['salary'].str.contains('\d',na=False)]['salary']
    df['salary']= df[df['salary'].str.contains('\d',na=False)]['salary'].str.replace(' to ','-',regex=True)
    df['salary']= df[df['salary'].str.contains('\d',na=False)]['salary'].str.replace('\d\%','',regex=True)
    df['salary']= df[df['salary'].str.contains('\d',na=False)]['salary'].str.replace(',','',regex=True)
    df['salary']= df[df['salary'].str.contains('\d',na=False)]['salary'].str.replace(' ','',regex=True)
    df['salary'].fillna('unknown', inplace=True)
    df[['low_salary','high_salary']] = df['salary'].str.split('-', n=1,expand=True).fillna('unknown')
    df['low_salary'] = df['low_salary'].str.extract('(\d+\.\d+|\d+ \d+|\d+k|\d+)')
    df['high_salary'] = df['high_salary'].str.extract('(\d+\.\d+|\d+ \d+|\d+k|\d+)')
    df['low_salary'] = df[df['low_salary'].str.contains('\d',na=False)]['low_salary'].str.replace('k', '000',regex=True)
    df['high_salary'] = df[df['high_salary'].str.contains('\d',na=False)]['high_salary'].str.replace('k', '000',regex=True)
    df['low_salary'] = df['low_salary'].fillna('unknown')
    df['high_salary'] = df['high_salary'].fillna('unknown')
    df.loc[df['low_salary'] == 'unknown', 'low_salary'] = df['high_salary']
    df.loc[df['high_salary'] == 'unknown', 'high_salary'] = df['low_salary']

    for r, row in enumerate(df['low_salary'].values):
        if row == 'unknown':
            df['low_salary'][r] = np.random.randint(salary_floor, salary_ceiling, size=1)[0]

    for r, row in enumerate(df['high_salary'].values):
        if row == 'unknown':
            df['high_salary'][r] = max(np.random.randint(salary_floor, salary_ceiling, size=1)[0], df['low_salary'][r])
    df['low_salary'] = df['low_salary'].astype('float32')
    df['high_salary'] = df['high_salary'].astype('float32')
    df.loc[df['low_salary'] < 50, 'low_salary'] =df['low_salary'] * 8*200
    df.loc[df['high_salary'] < 50, 'high_salary'] =df['high_salary'] * 8*200
    #threshold = (45000+75000) /2
    #df['lower_salary'] = np.random.randint(45000, threshold, df.shape[0])
    #df['upper_salary'] = np.random.randint(threshold, 75000, df.shape[0])

    return df



In [118]:
def select_column(df):
    df[['company_name', 'posted_time','ad_type','listing_time_indays','location','area','Classification','low_salary','high_salary']]
    df

In [116]:
Admin_cleaning = (df.pipe(pre_cleaning).
    pipe(listing_time_cleaning).
    pipe(location_cleaning).
    pipe(classification_cleaning).
    pipe(random_salary,45000,75000).#salary floor and ceiling
    pipe(select_column)
)


In [117]:
df = load_data("NZ_Admin_JOBS.xlsx") #input the file name
Admin_cleaning

,0,1,2,3,4,5,company_name,posted_time,featured_at,ad_type,listing_time_indays,location,area,subclassification,Classification,mainClassification,7,salary,low_salary,high_salary
0,Administrator,https://www.seek.co.nz/job/50582301?type=promo...,NaN,location: Bay of PlentyBay of Plentyarea: Taur...,"Featured,at,Private Advertiser",classification: Administration & Office Suppor...,Private Advertiser,Featured,Private Advertiser,promoted,0,Bay of Plenty,Tauranga,Office Support,Office Management,Administration,Office SupportAdministration,unknown,47648.0,66440.0
1,Receptionist,https://www.seek.co.nz/job/50620889?type=promo...,Avenues Orthodontics,location: Bay of PlentyBay of Plentyarea: Taur...,"Featured,at",classification: Administration & Office Suppor...,Avenues Orthodontics,Featured,None,promoted,0,Bay of Plenty,Tauranga,Office Support,Receptionists,Administration,Office SupportAdministration,unknown,51021.0,56294.0
2,Prosecutions Support Officer,https://www.seek.co.nz/job/50622169?type=stand...,New Zealand Police,location: AucklandAuckland,"4d ago,at",classification: Administration & Office Suppor...,New Zealand Police,4d ago,None,standard,4,Auckland,unknown,Office Support,Other,Administration,Office SupportAdministration,unknown,50648.0,50648.0
3,Early Childhood Centre Administrator,https://www.seek.co.nz/job/50639620?type=stand...,Kew Pacific Island Early Learning Centre,location: SouthlandSouthlandarea: Invercargill...,"1h ago,at",classification: Administration & Office Suppor...,Kew Pacific Island Early Learning Centre,1h ago,None,standard,0,Southland,Invercargill,Office Support,Administrative Assistants,Administration,Office SupportAdministration,unknown,61763.0,61763.0
4,Business Support Administrator,https://www.seek.co.nz/job/50622432?type=stand...,NaN,location: CanterburyCanterburyarea: Christchur...,"4d ago,at,Private Advertiser",classification: Administration & Office Suppor...,Private Advertiser,4d ago,Private Advertiser,standout,4,Canterbury,Christchurch,Office Support,Client & Sales Administration,Administration,Office SupportAdministration,unknown,51306.0,70829.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2703,Key Account Manager,https://www.seek.co.nz/job/50490062?type=stand...,Hays Talent Solutions,location: AucklandAucklandarea: Auckland Centr...,"27d ago,at",classification: Administration & Office Suppor...,Hays Talent Solutions,27d ago,None,standout,27,Auckland,Auckland Central,Office Support,Client & Sales Administration,Administration,Office SupportAdministration,unknown,64498.0,64498.0
2704,Executive Assistant,https://www.seek.co.nz/job/50488000?type=stand...,one eighty recruitment,location: WellingtonWellingtonarea: Wellington...,"27d ago,at",Competitive hourly rate,one eighty recruitment,27d ago,None,standout,27,Wellington,"Wellington CentralWellington Central,Competiti...",unknown,unknown,rly rate,None,unknown,65633.0,65633.0
2705,Temporary Office Roles,https://www.seek.co.nz/job/50524865?type=stand...,Asset Recruitment Ltd,location: WaikatoWaikatoarea: HamiltonHamilton...,"20d ago,at",Competitive hourly rates $$,Asset Recruitment Ltd,20d ago,None,standout,20,Waikato,"HamiltonHamilton,Competitive hourly rates $$",unknown,unknown,rly rates $$,None,unknown,53364.0,59318.0
2706,Temporary Office Roles,https://www.seek.co.nz/job/50477118?type=stand...,Asset Recruitment Ltd,location: WaikatoWaikatoarea: HamiltonHamilton...,"28d ago,at",Competitive hourly rates $$,Asset Recruitment Ltd,28d ago,None,standout,28,Waikato,"HamiltonHamilton,Competitive hourly rates $$",unknown,unknown,rly rates $$,None,unknown,61685.0,61685.0


,0,1,2,3,4,5
0,Administrator,https://www.seek.co.nz/job/50582301?type=promo...,NaN,location: Bay of PlentyBay of Plentyarea: Taur...,"Featured,at,Private Advertiser",classification: Administration & Office Suppor...
1,Receptionist,https://www.seek.co.nz/job/50620889?type=promo...,Avenues Orthodontics,location: Bay of PlentyBay of Plentyarea: Taur...,"Featured,at",classification: Administration & Office Suppor...
2,Prosecutions Support Officer,https://www.seek.co.nz/job/50622169?type=stand...,New Zealand Police,location: AucklandAuckland,"4d ago,at",classification: Administration & Office Suppor...
3,Early Childhood Centre Administrator,https://www.seek.co.nz/job/50639620?type=stand...,Kew Pacific Island Early Learning Centre,location: SouthlandSouthlandarea: Invercargill...,"1h ago,at",classification: Administration & Office Suppor...
4,Business Support Administrator,https://www.seek.co.nz/job/50622432?type=stand...,NaN,location: CanterburyCanterburyarea: Christchur...,"4d ago,at,Private Advertiser",classification: Administration & Office Suppor...
...,...,...,...,...,...,...
2703,Key Account Manager,https://www.seek.co.nz/job/50490062?type=stand...,Hays Talent Solutions,location: AucklandAucklandarea: Auckland Centr...,"27d ago,at",classification: Administration & Office Suppor...
2704,Executive Assistant,https://www.seek.co.nz/job/50488000?type=stand...,one eighty recruitment,location: WellingtonWellingtonarea: Wellington...,"27d ago,at",Competitive hourly rate
2705,Temporary Office Roles,https://www.seek.co.nz/job/50524865?type=stand...,Asset Recruitment Ltd,location: WaikatoWaikatoarea: HamiltonHamilton...,"20d ago,at",Competitive hourly rates $$
2706,Temporary Office Roles,https://www.seek.co.nz/job/50477118?type=stand...,Asset Recruitment Ltd,location: WaikatoWaikatoarea: HamiltonHamilton...,"28d ago,at",Competitive hourly rates $$
